# TyDI Question Generation: Inference example

In this notebook, we will show how to use a pretrained multilingual PassageQG model to generate questions. Given a text snippet, stanza is used to identify noun chunks (named entities) which becomes the answer and an mT5 is used to generate question givent he answer and the text snippet.

## Dependencies

If not already done, make sure to install PrimeQA with notebooks extras before getting started.

In [2]:
from primeqa.qg.models.qg_model import QGModel

2023-11-19 15:41:27.613210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-19 15:41:27.613244: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


{"time":"2023-11-19 15:41:29,299", "name": "numexpr.utils", "level": "INFO", "message": "Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8."}
{"time":"2023-11-19 15:41:29,300", "name": "numexpr.utils", "level": "INFO", "message": "NumExpr defaulting to 8 threads."}


{"time":"2023-11-19 15:41:30,406", "name": "stanza", "level": "INFO", "message": "Downloading default packages for language: multilingual (multilingual) ..."}
{"time":"2023-11-19 15:41:30,432", "name": "stanza", "level": "INFO", "message": "File exists: /home/sjbuhan/stanza_resources/multilingual/default.zip"}
{"time":"2023-11-19 15:41:30,520", "name": "stanza", "level": "INFO", "message": "Finished downloading models and saved to /home/sjbuhan/stanza_resources."}


## Loading pretrained model from huggingface

This model was trained using PrimeQA library and uploaded to huggingface hub.

In [3]:
model_name = 'PrimeQA/mt5-base-tydi-question-generator'
passage_qg_model = QGModel(model_name, modality='passage')

Loaded NER model for  Arabic
Loaded NER model for  English
Loaded NER model for  Finnish
Loaded NER model for  Russian


<br>

## Sample instance

Passages should be passed a `list` of `str`. We take one English and one Russian text to generate questions.

In [4]:
text_list = ["""
Joseon, officially Great Joseon State, was the last dynastic kingdom of Korea, lasting just over 500 years. It was founded by Yi Seong-gye in July 1392 and replaced by the Korean Empire in October 1897. The kingdom was founded following the aftermath of the overthrow of Goryeo in what is today the city of Kaesong. Early on, Korea was retitled and the capital was relocated to modern-day Seoul. The kingdom's northernmost borders were expanded to the natural boundaries at the rivers of Amrok and Tuman through the subjugation of the Jurchens.
""",
             "This is test data. And Django is a trash"
            ]

id_list = [
    """Joseon""","test"
]

In [5]:
import os

path = "./Data"
ids = os.listdir(path)

print(ids)

['Sejo_of_Joseon', 'Prince_Jeongwon', 'Grand_Prince_Neungchang', 'Yejong_of_Joseon', 'Jeongye_Daewongun', 'Danjong_of_Joseon', 'Prince_Yeollyeong', 'Jeongjo_of_Joseon', 'Crown_Prince_Sado', 'Crown_Prince_Hyojang', 'Grand_Prince_Inpyeong', 'Gyeongjong_of_Joseon', 'Crown_Prince_Uigyeong', 'Jungjong_of_Joseon', 'Myeongjong_of_Joseon', 'Sejong_the_Great', 'Prince_Euneon', 'Heonjong_of_Joseon', 'Injo_of_Joseon', 'Gwanghaegun_of_Joseon', 'Jeongjong_of_Joseon', 'Seonjo_of_Joseon', 'Deokheung_Daewongun', 'Injong_of_Joseon', 'Taejong_of_Joseon', 'Cheoljong_of_Joseon', 'Sunjo_of_Joseon', 'Joseon', 'Hyojong_of_Joseon', 'Munjong_of_Joseon', '.DS_Store', 'Seongjong_of_Joseon', 'Sukjong_of_Joseon', 'Heungseon_Daewongun', 'Yeongjo_of_Joseon', 'Hyeonjong_of_Joseon', 'Taejo_of_Joseon', 'Yeonsangun_of_Joseon', 'Crown_Prince_Hyomyeong', 'Prince_Eunsin']


In [6]:
root_ids = [
    "Taejo_of_Joseon",
    "Jeongjong_of_Joseon",
    "Taejong_of_Joseon",
    "Sejong_the_Great",
    "Munjong_of_Joseon",
]

In [7]:
def id_to_path(id):
    return f'{path}/{id}/iter0/{id}'

files = list(map(id_to_path, root_ids))
print(files)

['./Data/Taejo_of_Joseon/iter0/Taejo_of_Joseon', './Data/Jeongjong_of_Joseon/iter0/Jeongjong_of_Joseon', './Data/Taejong_of_Joseon/iter0/Taejong_of_Joseon', './Data/Sejong_the_Great/iter0/Sejong_the_Great', './Data/Munjong_of_Joseon/iter0/Munjong_of_Joseon']


In [28]:
import re

ids = []
texts = []

for root_id in root_ids:
    file_path = id_to_path(root_id)
    print(file_path)
    
    with open(file_path, "r") as f:
        text = f.read()
        text = re.sub(r" ?\([^)]+\)", "", text)
        text = re.sub(r" ?\{[^}]+\}", "", text)
        text = re.sub(r" ?\[[^]]+\]", "", text)
        
        
        for subtext in ("Abstract\n" + text).split("\n\n"):
            id = root_id + ' - ' + subtext.split("\n")[0]
            body = "\n".join(subtext.split("\n")[1:])
            
            if len(id) == 0 or len(body) < 20:
                continue
            ids.append(id)
            texts.append(body)
            
            print(f' # {id}')
            print(body)

./Data/Taejo_of_Joseon/iter0/Taejo_of_Joseon
 # Taejo_of_Joseon - Abstract
Taejo of Joseon, personal name Yi Seong-gye, later Yi Dan, was the founder and first ruler of the Joseon dynasty of Korea. After overthrowing the Goryeo dynasty, he ascended to the throne in 1392 and abdicated six years later during a strife between his sons. He was honored as Emperor Go following the establishment of the Korean Empire.
Taejo emphasized continuity over change. No new institutions were created and no massive purges occurred during his reign. His new dynasty was largely dominated by the same ruling families and officials that had served the previous regime. He re-established amicable ties with Japan and improved relations with Ming China.
 # Taejo_of_Joseon - Biography
Early life
Taejo's father was Yi Ja-chun, an official of Korean ethnicity serving the Mongol-led Yuan dynasty. His mother, Lady Choe, came from a prominent family originally from Deungju; her father, another Yuan official of Korean 

## Generate questions

The `generate_questions` function can take two arguments.
#### Controls:
- `num_questions_per_instance`: Number of questions to generate per table (default=5)
- `answers_list`: Generated questions will have these as the answers. It should be a list of lists, 
        where each list corresponds a passage in `text_list`. (default=[])
- `id_list`: Include an id_list of context passages aligned with text_list, defaults to empty list.

When `answers_list` is not provided, named entity recognition method is used to sample answers.

In [30]:
text_list = texts
id_list = ids

results = []
errors = []

for (text, id) in zip(text_list, id_list):
    print(f' # {id}')
    print(text)

    try:
        results += passage_qg_model.generate_questions([text], 
                        num_questions_per_instance = 20, id_list=[id])
    except Exception as error:
        print(error)
        errors.append({
            "id": id,
            "text": text,
            "error": error.__str__(),
        })

print(len(results))
print(len(errors))

 # Taejo_of_Joseon - Abstract
Taejo of Joseon, personal name Yi Seong-gye, later Yi Dan, was the founder and first ruler of the Joseon dynasty of Korea. After overthrowing the Goryeo dynasty, he ascended to the throne in 1392 and abdicated six years later during a strife between his sons. He was honored as Emperor Go following the establishment of the Korean Empire.
Taejo emphasized continuity over change. No new institutions were created and no massive purges occurred during his reign. His new dynasty was largely dominated by the same ruling families and officials that had served the previous regime. He re-established amicable ties with Japan and improved relations with Ming China.
Input language en
 # Taejo_of_Joseon - Biography
Early life
Taejo's father was Yi Ja-chun, an official of Korean ethnicity serving the Mongol-led Yuan dynasty. His mother, Lady Choe, came from a prominent family originally from Deungju; her father, another Yuan official of Korean ethnicity, rose to the posi

In [34]:
import json

with open('results.json', 'w') as f:
    json.dump(results, f)

with open('errors.json', 'w') as f:
    json.dump(errors, f)

['Sejong_the_Great - Literature', {'id': 'Sejong_the_Great - Literature', 'text': 'Sejong composed the famous Yongbieocheonga, Seokbo Sangjeol, Worin Cheongang Jigok, and Dongguk Jeongun.', 'error': NotImplementedError()}, 'Sejong_the_Great - See also', {'id': 'Sejong_the_Great - See also', 'text': 'King Sejong Institute\nKing Sejong Station\nSejong Center\nSejong City\nSejong the Great-class destroyer\nSejongno\nUNESCO King Sejong Literacy Prize\nHunminjeongeum\nHangul', 'error': NotImplementedError()}, 'Munjong_of_Joseon - External links', {'id': 'Munjong_of_Joseon - External links', 'text': '<http://navercast.naver.com/contents.nhn?rid=77&contents_id=57444>\n', 'error': NotImplementedError('This language is not supported: ja')}]


TypeError: string indices must be integers

Answer sampler only supports Arabic, English, Finnish and Russian now. For other languages in TyDi dataset
we should provide the answers explicitly.

In [7]:
text_list = ["세종대왕은 조선시대의 왕이다."]
# text_list = ["শচীন টেন্ডুলকারকে ক্রিকেট ইতিহাসের অন্যতম সেরা ব্যাটসম্যান হিসেবে গণ্য করা হয়।"]
# answers_list = [["শচীন টেন্ডুলকার"]]
passage_qg_model.generate_questions(text_list)

Input language ko


NotImplementedError: 